Importing the libraries needed for the project, and reading in the first dataset (boardgames_ranks.csv)

In [ ]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
import xml.etree.ElementTree as ET
import time
from bs4 import BeautifulSoup
import re

df_board_game_rankings = pd.read_csv('.\\.venv\\data\\boardgames_ranks.csv')
df_board_game_rankings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158945 entries, 0 to 158944
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   158945 non-null  int64  
 1   name                 158945 non-null  object 
 2   yearpublished        158945 non-null  int64  
 3   rank                 158945 non-null  int64  
 4   bayesaverage         158945 non-null  float64
 5   average              158945 non-null  float64
 6   usersrated           158945 non-null  int64  
 7   is_expansion         158945 non-null  int64  
 8   abstracts_rank       1424 non-null    float64
 9   cgs_rank             362 non-null     float64
 10  childrensgames_rank  1073 non-null    float64
 11  familygames_rank     3314 non-null    float64
 12  partygames_rank      912 non-null     float64
 13  strategygames_rank   2992 non-null    float64
 14  thematic_rank        1671 non-null    float64
 15  wargames_rank    

Filter out games with rank of 0, as we only want games that have been ranked.

In [160]:
df_bgr2 = pd.read_csv('.\\.venv\\data\\boardgames_ranks.csv')
zerorank = df_bgr2[(df_bgr2['rank'] == 0)].index
df_bgr_nozero = pd.DataFrame(df_bgr2.drop(zerorank, inplace = True))
df_bgr2.head(10)

,id,name,yearpublished,rank,bayesaverage,average,usersrated,is_expansion,abstracts_rank,cgs_rank,childrensgames_rank,familygames_rank,partygames_rank,strategygames_rank,thematic_rank,wargames_rank
0,224517,Brass: Birmingham,2018,1,8.41277,8.59213,48001,0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1,161936,Pandemic Legacy: Season 1,2015,2,8.37571,8.52498,54148,0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN
2,174430,Gloomhaven,2017,3,8.34398,8.57972,63128,0,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN
3,342942,Ark Nova,2021,4,8.33959,8.53505,46382,0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
4,233078,Twilight Imperium: Fourth Edition,2017,5,8.23662,8.59240,24556,0,NaN,NaN,NaN,NaN,NaN,5.0,3.0,NaN
5,316554,Dune: Imperium,2020,6,8.23137,8.43403,47922,0,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN
6,167791,Terraforming Mars,2016,7,8.20734,8.35483,101433,0,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN
7,115746,War of the Ring: Second Edition,2011,8,8.19064,8.54570,22008,0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1.0
8,187645,Star Wars: Rebellion,2016,9,8.16964,8.41835,33163,0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
9,291457,Gloomhaven: Jaws of the Lion,2020,10,8.15191,8.42294,35486,0,NaN,NaN,NaN,NaN,NaN,9.0,7.0,NaN


Pulling only the top 250 overall rank from boardgames_ranks.csv

In [161]:
df_bgr_top = pd.DataFrame(df_bgr2.loc[(df_bgr2['rank'] >=1) & (df_bgr2['rank'] <=250)])
df_bgr_top.head(20)

,id,name,yearpublished,rank,bayesaverage,average,usersrated,is_expansion,abstracts_rank,cgs_rank,childrensgames_rank,familygames_rank,partygames_rank,strategygames_rank,thematic_rank,wargames_rank
0,224517,Brass: Birmingham,2018,1,8.41277,8.59213,48001,0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1,161936,Pandemic Legacy: Season 1,2015,2,8.37571,8.52498,54148,0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN
2,174430,Gloomhaven,2017,3,8.34398,8.57972,63128,0,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN
3,342942,Ark Nova,2021,4,8.33959,8.53505,46382,0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
4,233078,Twilight Imperium: Fourth Edition,2017,5,8.23662,8.59240,24556,0,NaN,NaN,NaN,NaN,NaN,5.0,3.0,NaN
5,316554,Dune: Imperium,2020,6,8.23137,8.43403,47922,0,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN
6,167791,Terraforming Mars,2016,7,8.20734,8.35483,101433,0,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN
7,115746,War of the Ring: Second Edition,2011,8,8.19064,8.54570,22008,0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1.0
8,187645,Star Wars: Rebellion,2016,9,8.16964,8.41835,33163,0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
9,291457,Gloomhaven: Jaws of the Lion,2020,10,8.15191,8.42294,35486,0,NaN,NaN,NaN,NaN,NaN,9.0,7.0,NaN


Replacing NaN values in the DataFrame with '0'

In [162]:
df_bgr_top.fillna(0, inplace=True)


Converting rank fields to int.

In [163]:
convert_dict = {'abstracts_rank': int,
                'cgs_rank': int,
                'childrensgames_rank': int,
                'familygames_rank': int,
                'partygames_rank': int,
                'strategygames_rank': int,
                'thematic_rank': int,
                'wargames_rank': int}
df_bgr_top = df_bgr_top.astype(convert_dict)
print(df_bgr_top.dtypes)

id                       int64
name                    object
yearpublished            int64
rank                     int64
bayesaverage           float64
average                float64
usersrated               int64
is_expansion             int64
abstracts_rank           int64
cgs_rank                 int64
childrensgames_rank      int64
familygames_rank         int64
partygames_rank          int64
strategygames_rank       int64
thematic_rank            int64
wargames_rank            int64
dtype: object


In [47]:
df_bgr_top.info()

<class 'pandas.core.frame.DataFrame'>
Index: 250 entries, 0 to 249
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   250 non-null    int64  
 1   name                 250 non-null    object 
 2   yearpublished        250 non-null    int64  
 3   rank                 250 non-null    int64  
 4   bayesaverage         250 non-null    float64
 5   average              250 non-null    float64
 6   usersrated           250 non-null    int64  
 7   is_expansion         250 non-null    int64  
 8   abstracts_rank       250 non-null    int64  
 9   cgs_rank             250 non-null    int64  
 10  childrensgames_rank  250 non-null    int64  
 11  familygames_rank     250 non-null    int64  
 12  partygames_rank      250 non-null    int64  
 13  strategygames_rank   250 non-null    int64  
 14  thematic_rank        250 non-null    int64  
 15  wargames_rank        250 non-null    int64  


Rounding averages to 2 digits.

In [48]:
df_bgr_top.bayesaverage = df_bgr2.bayesaverage.round(2)
df_bgr_top.average = df_bgr2.average.round(2)
df_bgr_top.head(20)

,id,name,yearpublished,rank,bayesaverage,average,usersrated,is_expansion,abstracts_rank,cgs_rank,childrensgames_rank,familygames_rank,partygames_rank,strategygames_rank,thematic_rank,wargames_rank
0,224517,Brass: Birmingham,2018,1,8.41,8.59,48001,0,0,0,0,0,0,1,0,0
1,161936,Pandemic Legacy: Season 1,2015,2,8.38,8.52,54148,0,0,0,0,0,0,2,1,0
2,174430,Gloomhaven,2017,3,8.34,8.58,63128,0,0,0,0,0,0,4,2,0
3,342942,Ark Nova,2021,4,8.34,8.54,46382,0,0,0,0,0,0,3,0,0
4,233078,Twilight Imperium: Fourth Edition,2017,5,8.24,8.59,24556,0,0,0,0,0,0,5,3,0
5,316554,Dune: Imperium,2020,6,8.23,8.43,47922,0,0,0,0,0,0,6,0,0
6,167791,Terraforming Mars,2016,7,8.21,8.35,101433,0,0,0,0,0,0,7,0,0
7,115746,War of the Ring: Second Edition,2011,8,8.19,8.55,22008,0,0,0,0,0,0,0,4,1
8,187645,Star Wars: Rebellion,2016,9,8.17,8.42,33163,0,0,0,0,0,0,0,5,0
9,291457,Gloomhaven: Jaws of the Lion,2020,10,8.15,8.42,35486,0,0,0,0,0,0,9,7,0


Outputting to csv

In [149]:
df_bgr_top.to_csv('.\\.venv\\data\\bgr_top.csv', index=False)

Extract BGG IDs from csv into list to feed into API.

In [164]:
bg_ids = df_bgr_top['id'].values.tolist()
print(bg_ids)




[224517, 161936, 174430, 342942, 233078, 316554, 167791, 115746, 187645, 291457, 162886, 220308, 12333, 182028, 193738, 84876, 169786, 246900, 173346, 28720, 167355, 266507, 177736, 124361, 295770, 341169, 205637, 312484, 266192, 120677, 237182, 192135, 164928, 397598, 96848, 251247, 324856, 199792, 183394, 366013, 285774, 175914, 256960, 247763, 284378, 3076, 521, 295947, 102794, 185343, 184267, 314040, 170216, 31260, 255984, 161533, 253344, 221107, 373106, 251661, 205059, 231733, 321608, 126163, 182874, 2651, 276025, 244521, 216132, 35677, 266810, 125153, 164153, 124742, 209010, 284083, 55690, 200680, 230802, 28143, 157354, 383179, 72125, 201808, 191189, 159675, 229853, 110327, 317985, 25613, 93, 62219, 365717, 68448, 121921, 171623, 256916, 279537, 332772, 366161, 291453, 225694, 264220, 155821, 37111, 236457, 163068, 322289, 310873, 122515, 170042, 42, 40834, 364073, 18602, 203993, 73439, 146021, 271055, 172386, 277659, 269385, 217372, 371942, 180263, 12493, 163412, 144733, 102680,

Experimenting with Parsing Options

In [172]:
#parsing
#parser = etree.XMLParser(recover=True)
response_API_test = requests.get('https://boardgamegeek.com/xmlapi2/thing?id=224517')
api_data = response_API_test.content
root = ET.fromstring(api_data)
root[0].find('description').text
##print(root.tag)
#for child in root:
#   print(child.tag, child.attrib)

#for elem in root.iter('type'):
#    print(elem.text)

##game_xml_test = BeautifulSoup(response_API_test.text, 'lxml-xml')
##print(game_xml_test)


"Brass: Birmingham is an economic strategy game sequel to Martin Wallace's 2007 masterpiece, Brass. Brass: Birmingham tells the story of competing entrepreneurs in Birmingham during the industrial revolution between the years of 1770 and 1870.&#10;&#10;It offers a very different story arc and experience from its predecessor. As in its predecessor, you must develop, build and establish your industries and network in an effort to exploit low or high market demands. The game is played over two halves: the canal era (years 1770-1830) and the rail era (years 1830-1870). To win the game, score the most VPs. VPs are counted at the end of each half for the canals, rails and established (flipped) industry tiles.&#10;&#10;Each round, players take turns according to the turn order track, receiving two actions to perform any of the following actions (found in the original game):&#10;&#10;1) Build - Pay required resources and place an industry tile.&#10;2) Network - Add a rail / canal link, expandi

Small scale test of XML Parsing based on one BGG ID.

In [207]:
root = ET.fromstring(api_data)
bgg_id_ls = []
des_id_ls = []
des_name_ls = []
##designers = {'BGG ID': bgg_id_ser}
designers_smol = {'BGG_ID': ['224517']}

##for index, row in df_designersv3.iterrows(): 
for item in root:
  for link in item.findall('link'):
    if(link.get('type') == 'boardgamedesigner'):
      des_id_ls.append(link.get('id'))
      des_name_ls.append(link.get('value'))
      bgg_id_ls.append(item.get('id'))
des_id_ser = pd.Series(des_id_ls)
des_name_ser = pd.Series(des_name_ls)
bgg_id_ser = pd.Series(bgg_id_ls)
desv3_frame = {'BGG_ID': bgg_id_ser, 'Designer_ID': des_id_ser, 'Designer_Name': des_name_ser}
df_designersv3 = pd.DataFrame(desv3_frame)
df_designersv3


,BGG_ID,Designer_ID,Designer_Name
0,224517,32887,Gavan Brown
1,224517,32943,Matt Tolman
2,224517,6,Martin Wallace


Larger Scale Test

In [218]:
df_bgr_30 = pd.DataFrame(df_bgr2.loc[(df_bgr2['rank'] >=1) & (df_bgr2['rank'] <=30)])
bg_ids_30 = df_bgr_30['id'].values.tolist()
bgidtest30 = [i for (i,x) in enumerate(bg_ids_30) if x%4 == 0]
for id in bg_ids_30:
    if id%4 == 0:
        print(id)

161936
220308
182028
84876
246900
28720
177736
312484
266192


In [227]:
API_base_string = 'https://boardgamegeek.com/xmlapi2/thing?id='
bgg_id_ls = []
des_id_ls = []
des_name_ls = []
for id in bg_ids_30:
    api_rec_30 = requests.get(API_base_string + str(id))
    api_data_30 = api_rec_30.content
    root = ET.fromstring(api_data_30)
    if id%4 == 0: 
      time.sleep(20)
      for item in root:
        for link in item.findall('link'):
          if(link.get('type') == 'boardgamedesigner'):
            des_id_ls.append(link.get('id'))
            des_name_ls.append(link.get('value'))
            bgg_id_ls.append(item.get('id'))
    else:
      for item in root:
        for link in item.findall('link'):
          if(link.get('type') == 'boardgamedesigner'):
            des_id_ls.append(link.get('id'))
            des_name_ls.append(link.get('value'))
            bgg_id_ls.append(item.get('id'))
         
des_id_ser = pd.Series(des_id_ls)
des_name_ser = pd.Series(des_name_ls)
bgg_id_ser = pd.Series(bgg_id_ls)
desv3_frame = {'BGG_ID': bgg_id_ser, 'Designer_ID': des_id_ser, 'Designer_Name': des_name_ser}
df_designersv3 = pd.DataFrame(desv3_frame)
df_designersv3



,BGG_ID,Designer_ID,Designer_Name
0,224517,32887,Gavan Brown
1,224517,32943,Matt Tolman
2,224517,6,Martin Wallace
3,161936,442,Rob Daviau
4,161936,378,Matt Leacock
5,174430,69802,Isaac Childres
6,342942,138517,Mathias Wigge
7,233078,96049,Dane Beltrami
8,233078,6651,Corey Konieczka
9,233078,21,Christian T. Petersen


Full Run of 250 BGGIDS, pulling designer information

In [ ]:
API_base_string = 'https://boardgamegeek.com/xmlapi2/thing?id='
bgg_id_ls = []
des_id_ls = []
des_name_ls = []
for id in bg_ids:
    api_rec_full = requests.get(API_base_string + str(id))
    api_data_full = api_rec_full.content
    root = ET.fromstring(api_data_full)
    if id%2 == 0: 
      time.sleep(15)
      for item in root:
        for link in item.findall('link'):
          if(link.get('type') == 'boardgamedesigner'):
            des_id_ls.append(link.get('id'))
            des_name_ls.append(link.get('value'))
            bgg_id_ls.append(item.get('id'))
    else:
      for item in root:
        for link in item.findall('link'):
          if(link.get('type') == 'boardgamedesigner'):
            des_id_ls.append(link.get('id'))
            des_name_ls.append(link.get('value'))
            bgg_id_ls.append(item.get('id'))
         
des_id_ser = pd.Series(des_id_ls)
des_name_ser = pd.Series(des_name_ls)
bgg_id_ser = pd.Series(bgg_id_ls)
desv3_frame = {'BGG_ID': bgg_id_ser, 'Designer_ID': des_id_ser, 'Designer_Name': des_name_ser}
df_designersv3 = pd.DataFrame(desv3_frame)
df_designersv3.to_csv('.\\.venv\\data\\bgg_designer_api_pull.csv', index=False)
df_designersv3

Need to Tweak to pull Mechanic information by BGG_ID

In [229]:
mech_id_ls = []
mech_name_ls = []
mech_bggid_ls = []

for id in bg_ids_30:
    api_rec_30 = requests.get(API_base_string + str(id))
    api_data_30 = api_rec_30.content
    root = ET.fromstring(api_data_30)
    if id%4 == 0: 
      time.sleep(8)
      for item in root:
        for link in item.findall('link'):
          if(link.get('type') == 'boardgamemechanic'):
            mech_id_ls.append(link.get('id'))
            mech_name_ls.append(link.get('value'))
            mech_bggid_ls.append(item.get('id'))
    else:
      for item in root:
        for link in item.findall('link'):
           if(link.get('type') == 'boardgamemechanic'):
            mech_id_ls.append(link.get('id'))
            mech_name_ls.append(link.get('value'))
            mech_bggid_ls.append(item.get('id'))
         
mech_bggid_ser = pd.Series(mech_bggid_ls)
mech_id_ser = pd.Series(mech_id_ls)
mech_name_ser = pd.Series(mech_name_ls)
mechv3_frame = {'BGG_ID': mech_bggid_ser, 'Mechanic_ID': mech_id_ser, 'Mechanic_Name': mech_name_ser}
df_mechv3 = pd.DataFrame(mechv3_frame)
df_mechv3

,BGG_ID,Mechanic_ID,Mechanic_Name
0,224517,2040,Hand Management
1,224517,2902,Income
2,224517,2904,Loans
3,224517,2900,Market
4,224517,3099,Multi-Use Cards
...,...,...,...
382,120677,2849,Tech Trees / Tech Tracks
383,120677,2830,Turn Order: Pass Order
384,120677,2015,Variable Player Powers
385,120677,2897,Variable Set-up


In [180]:
response_API_test = requests.get('https://boardgamegeek.com/xmlapi2/thing?id=224517')
api_data = response_API_test.content

##json_bgame_data = xmltodict.parse(api_data)
soup = BeautifulSoup(api_data, 'html.parser')
print(soup.prettify())

<?xml version="1.0" encoding="utf-8"?>
<items termsofuse="https://boardgamegeek.com/xmlapi/termsofuse">
 <item id="224517" type="boardgame">
  <thumbnail>
   https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg70Jgw__thumb/img/o18rjEemoWaVru9Y2TyPwuIaRfE=/fit-in/200x150/filters:strip_icc()/pic3490053.jpg
  </thumbnail>
  <image/>
  https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg70Jgw__original/img/FpyxH41Y6_ROoePAilPNEhXnzO8=/0x0/filters:format(jpeg)/pic3490053.jpg
  <name sortindex="1" type="primary" value="Brass: Birmingham">
  </name>
  <name sortindex="1" type="alternate" value="Brass. Бирмингем">
  </name>
  <name sortindex="1" type="alternate" value="Brass. Бірмінгем">
  </name>
  <name sortindex="1" type="alternate" value="ブラス：バーミンガム">
  </name>
  <name sortindex="1" type="alternate" value="工业革命：伯明翰(Chinese edition) (2018)">
  </name>
  <name sortindex="1" type="alternate" value="工業革命：伯明翰">
  </name>
  <name sortindex="1" type="alternate" value="브라스: 버밍엄">
  </name>
  <description>
   

C:\Program Files\Python312\Lib\html\parser.py:171: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


In [211]:
print('Start Time:', time.ctime())
bgidtest = [i for (i,x) in enumerate(bg_ids) if x%7 == 0]
for bg_ids in bgidtest:
    time.sleep(2)
print('End Time:', time.ctime())

Start Time: Wed Dec  4 20:17:21 2024
End Time: Wed Dec  4 20:18:15 2024


Iterating through Game Id List and sleeping for 5 seconds at intervals where board game id is divisible by 7. This is an arbitrary number and just set to adhere to the terms of use for the api.

In [ ]:

for i in [i for (i,x) in enumerate(bg_ids) if x%7 == 0]:
    time.sleep(5)
    response_API_XML2 = requests.get('https://boardgamegeek.com/xmlapi2/thing?id=' + i)
    game_id_xml = BeautifulSoup(response_API_XML2.text, 'xml')
    

    
#else:
   #response_API_XML2 = requests.get('https://boardgamegeek.com/xmlapi2/thing?id=' + i) 



In [ ]:
response_API = requests.get('https://boardgamegeek.com/xmlapi/boardgame/251696')
game_info_35424 = response_API.text

Read in Designer CSV File

In [ ]:
df_bgdesigner = pd.read_csv('.\\.venv\\data\\designers_reduced.csv')
df_bgdesigner.head()

Creating new dataframe from selection

In [ ]:
df_topbgdesigner = pd.DataFrame(df_bgdesigner.loc[df_bgdesigner['BGGId'].isin(bg_ids)])
df_topbgdesigner.reset_index(drop=True, inplace=True)
df_topbgdesigner.head(10)

Trying to select column names (designer name) from the designers_reduced.csv where BGGID is = to the list above and the value is equal to 1.

In [ ]:
df_designerv2 = pd.DataFrame()

for index, row in df_topbgdesigner:
    condition = (df_topbgdesigner.iloc[row] == 1)
    designerlst = df_topbgdesigner.columns[condition]
    print(designerlst)

Writing designers to CSV

In [ ]:
df_topbgdesigner.to_csv('.\\.venv\\data\\bgr_top_designer.csv', index=False)